# Image and text Extraction

In [2]:
from image_processor import extract_image
from text_processor import extract_text
from ultralytics import YOLO
from data_format import DataFormat

import json, os

# classes: 0-Caption 1-Handwritting 2-Image 3-Text

data_path = "../dataset/facticia_3_2/"
export_path = "../dataset/results/"

filename = "4c76a3e4-ohcbh_cf_erl8_204"

image_path = f"{data_path}/images/{filename}.jpg"

model = YOLO("detection/best.pt")

result = model(image_path)

obb = result[0].obb

xywhr = obb.xywhr
cls = obb.cls

data = []

# # Verify if the export folder exists
# if not os.path.exists(f"{export_path}/{filename}"):

# Create folder to store crops
os.makedirs(f"{export_path}/{filename}", exist_ok=True)

# Save crops and create metadata
for i in range(len(xywhr)):
    data.append(DataFormat(f"{filename}_{i}", xywhr[i], cls[i]))
    image = extract_image(image_path, xywhr[i])

    image.save(f"{export_path}/{filename}/{filename}_{i}.jpg")

for d in data:
    if d.type == 3:
        d.text = extract_text(f"{export_path}/{filename}/{d.filename}.jpg")
    

# Create json file to store crops metadata
with open(f"{export_path}/{filename}/{filename}.json", "w") as f:
    json.dump({d.filename: d.to_dict() for d in data}, f, indent=4)


image 1/1 /home/danieltm/Documents/MATCOM/4A 1S/ML/ML-Facticia/src/../dataset/facticia_3_2/images/4c76a3e4-ohcbh_cf_erl8_204.jpg: 1024x704 111.9ms
Speed: 6.1ms preprocess, 111.9ms inference, 1.5ms postprocess per image at shape (1, 3, 1024, 704)
LA REMODELACION
DE LA HABANA

Vista Genreal de la Alameda de Paula,
después de su restauración.

Vista general de la Plaza de Agua Dulce.

Vista del Paso Superior construido por

la Dirección Ceneral de Ingemieros del

Ministerio de Obras Públicas en la barriada
de Luyano.

- a a
SN A a

